In [7]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [8]:
total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 256
n_input = 28 * 28
n_noise = 128
X = tf.placeholder(tf.float32, [None, n_input]) # 실제 데이터 입력
Z = tf.placeholder(tf.float32, [None, n_noise]) # 노이즈 입력

In [9]:
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01)) # 128 x 256
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01)) # 256 x 784
G_b2 = tf.Variable(tf.zeros([n_input]))

D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

In [10]:
# 다른점1. cost function : 확률적 함수
# MSE, Entropy
# 다른점2. Kullback–Leibler divergence(쿨백 라이블러 발산) : GAN, VAE - 분포의 차를 확인하는 함수
# 생성기 -> noise가 input데이터, cost function: Kullback–Leibler divergence를 사용하는 것이 GAN의 특징

# 생성기
def generator(noise_z): # 128 x 128
    hidden = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1) # 128 x 256
    output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2) # 128 x 784
    return output # 784 이미지 생성

# 판별기 : 실제의 분포를 확인
def discriminator(inputs):
    hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)
    return output

def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

In [11]:
G = generator(Z) # 노이즈로 이미지를 생성한다.
D_gene = discriminator(G) # 이미지를 판별한다 => 분포 확인
D_real = discriminator(X) # 실제 이미지의 분포를 확인한다.
# log가 붙어있는 이유는 정보량을 구하는 것과 같다.
# [확률 증가, 정보량 감소], [확률 감소, 정보량 증가] *tfidf의 논리를 고려할 것
# 가장 적합한 분포를 찾아내는 것 => 높은 것이 유리하다.(최대값을 찾아내어야함)
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_gene))
loss_G = tf.reduce_mean(tf.log(D_gene)) # 최대우도 추정법 - 적합한 분포가 될 수 있도록
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]
# 최대화하기 위해?
# 역전파 변수를 지정
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list=G_var_list)

%matplotlib inline
sess = tf.Session()
sess.run(tf.global_variables_initializer())
total_batch = int(mnist.train.num_examples / batch_size)
loss_val_D, loss_val_G = 0, 0

In [16]:
# 이미지 분포의 특징을 추출하는 망
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        # 실제 이미지 분포 확인
        _, loss_val_D = sess.run([train_D, loss_D], feed_dict={X: batch_xs, Z: noise})
        # 이미지 생성 128x128로 데이터가 입력된다.
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict={Z: noise})
        
    print("Epoch:", "%04d" % epoch,
          "D loss: {:.4}".format(loss_val_D),
          "G loss: {:.4}".format(loss_val_G))
        
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        # 노이즈로부터 이미지를 생성 : generator
        noise = get_noise(sample_size, n_noise) # 10 x 128
        samples = sess.run(G, feed_dict={Z: noise})
        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size,1))
        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28,28)))
        plt.savefig("samples/{}.png".format(str(epoch).zfill(3)), bbox_inches="tight")
        plt.close(fig)
print("최적화 완료")

Epoch: 0000 D loss: -0.6941 G loss: -1.508
Epoch: 0001 D loss: -0.3959 G loss: -1.691
Epoch: 0002 D loss: -0.3785 G loss: -2.095
Epoch: 0003 D loss: -0.237 G loss: -2.99
Epoch: 0004 D loss: -0.3523 G loss: -2.472
Epoch: 0005 D loss: -0.2535 G loss: -2.566
Epoch: 0006 D loss: -0.3879 G loss: -2.263
Epoch: 0007 D loss: -0.3213 G loss: -2.43
Epoch: 0008 D loss: -0.3118 G loss: -2.73
Epoch: 0009 D loss: -0.2856 G loss: -2.781
Epoch: 0010 D loss: -0.2944 G loss: -2.568
Epoch: 0011 D loss: -0.3534 G loss: -2.68
Epoch: 0012 D loss: -0.3982 G loss: -2.588
Epoch: 0013 D loss: -0.3765 G loss: -2.504
Epoch: 0014 D loss: -0.3436 G loss: -2.533
Epoch: 0015 D loss: -0.3146 G loss: -2.605
Epoch: 0016 D loss: -0.384 G loss: -2.289
Epoch: 0017 D loss: -0.3569 G loss: -2.541
Epoch: 0018 D loss: -0.4609 G loss: -2.457
Epoch: 0019 D loss: -0.4095 G loss: -2.676
Epoch: 0020 D loss: -0.3917 G loss: -2.999
Epoch: 0021 D loss: -0.3401 G loss: -3.102
Epoch: 0022 D loss: -0.5079 G loss: -2.591
Epoch: 0023 D los